In [6]:
import time, pickle, ast, os
from tqdm import tqdm
import pandas as pd
import numpy as np

#script for reading/writing trees
from scripts.script import readP, writeTree
#script for build DTK
from scripts.script import createDTK
#script for parse sentences
from scripts.script import parse

from os import getcwd

import logging

logger = logging.getLogger(__name__)

print(getcwd());

E:\Tesi magistrale\KermitSimptc


In [7]:
# Download agnews
# !wget wget https://data.deepai.org/agnews.zip
#
# !unzip agnews.zip

data_train = pd.read_csv('./agnews/train.csv')
data_train = data_train.sample(1000)
dataset_name= "agnews"

In [8]:
name = 'dtk_trees_'+dataset_name+'.pkl'
name2 = 'log_'+dataset_name+'.txt'
name3 = 'dt_'+dataset_name+'.txt'

i = 0
cont = 0
listTree = []
newList = []
oldList = []

tree = "(S)"
treeException = createDTK(tree)


for line in tqdm(data_train['Description']):
    cont += 1
    try:
        tree = (parse(line))
        treeDTK = createDTK(tree)
    except Exception:
        tree, treeDTK = "(S)", treeException

    listTree.append(treeDTK)
    #write parenthetical tree
    writeTree(name3,tree)
    #every 5000 shafts saves the corresponding DTKs
    if i>5000:
        time.sleep(1)
        if os.path.isfile(name):
            #append new encoded tree in pickle file
            oldList = readP(name)
            newList = oldList+listTree
        else:
            newList = listTree

        f=open(name, 'wb')

        for x in newList:
            pickle.dump(x, f)
        f.close()

        f=open(name2, "a+")
        f.write(str(cont)+'..')
        f.close()

        i = 0
        listTree = []
        newList = []
        oldList = []
    else:
        i +=1

#checking consistency
if os.path.isfile(name):
    oldList = readP(name)
    newList = oldList+listTree
else:
    newList = listTree
f=open(name, 'wb')
for x in newList:
    pickle.dump(x, f)
f.close()

100%|██████████| 1000/1000 [05:14<00:00,  3.18it/s]


In [9]:
with open('dtk_trees_agnews.pkl', 'rb') as f:
    unpickled = pickle.load(f)


In [38]:
from simptc import configs
def encode_sentence(prompted_text):
    try:
        tree = (parse(prompted_text))
        treeDTK = createDTK(tree)
    except Exception:
        tree, treeDTK = "(S)", treeException
    return treeDTK

def load_label_names(dataset):
    label_name_file = f"./agnews/label_names.txt"
    labels = []
    with open(label_name_file, "r") as f:
        for line in f:
            words = line.split("\n")[0].split(",")
            labels.append(words)
    return labels

def encode_label_names(dataset, label_name_file, model, tokenizer, model_name):
    print("encoding class anchor sentences...")
    embedding_dir = f'./agnews/embeddings'
    if not os.path.exists(embedding_dir):
        os.mkdir(embedding_dir)
    simcse = False
    labels = []
    with open(label_name_file, "r") as f:
        for line in f:
            words = line.split("\n")[0].split(",")
            labels.append(words)

    template_number = len(configs.TEMPLATES["agnews"])
    for template_id in range(template_number):
        if simcse:
            model_name_with_prefix = f"simcse_{True}_" + model_name
        else:
            model_name_with_prefix = model_name
        if label_name_file.split('/')[-1] == 'label_names.txt':
            prompt_path = os.path.join(embedding_dir, f"prompt_{template_id}_{model_name_with_prefix}.npz")
        else:
            label_names_name = label_name_file.split('/')[-1][12:-4]   # extract label names name
            prompt_path = os.path.join(embedding_dir, f"prompt_{template_id}_{label_names_name}_{model_name_with_prefix}.npz")
        template = configs.TEMPLATES["agnews"][template_id]
        # create prompt embeddings:
        if not os.path.exists(prompt_path):
            print(f"generating prompt embeddings using {model_name} from {label_name_file}, template: {template}")
            print(f"embedding path = {prompt_path}")
            label_list = []
            prompt_embeddings = []
            for label_id, label_words in enumerate(labels):
                show_example = True
                for label_word in label_words:
                    prompted_text = template.replace("<mask>", label_word)
                    if show_example:
                        show_example = False
                        print(f"prompt exampe: {prompted_text}")
                    prompt_embeddings.append(encode_sentence(prompted_text))
                    label_list.append(label_id)
            prompt_embeddings = np.array(prompt_embeddings)
            label_list = np.array(label_list)
            print(f"embedding shape: {prompt_embeddings.shape}")
            np.savez(prompt_path, embedding_list=prompt_embeddings, label_list=label_list)
        else:
            print(f"prompt embeddings already exists at {prompt_path}")



In [43]:
def encode_and_save_sentences(dataset, suffix, model, tokenizer, model_name, split):
    logger.info("encoding train and test set...")
    # label_path = os.path.join(f"./datasets/TextClassification/{dataset}", f"{split}_labels.{suffix}")
    embedding_dir = os.path.join(f"./agnews/dataset/", "embeddings")

    if not os.path.exists(embedding_dir):
        os.mkdir(embedding_dir)
    if model_name in ["roberta_large_vanilla", "roberta_large_sentence"]:
        path = os.path.join(embedding_dir, f"{split}_{model_name}.npz")
        simcse = False
    else:
        path = os.path.join(embedding_dir, f"{split}_simcse_True_{model_name}.npz")
        simcse = True

    # create testing embeddings:
    embedding_path = path
    # data_path = os.path.join(f"./datasets/TextClassification/{dataset}", f"{split}.{suffix}")
    if not os.path.exists(embedding_path):
        logger.info(f"generating train/test embeddings using {model_name} from kermit")
        embedding_list, label_list = unpickled, [0,1,2,4]
        label_list = np.array(label_list)
        embedding_list = np.array(embedding_list)
        logger.info(f"embedding size: {embedding_list.shape}, label shape: {label_list.shape}")
        logger.info(f"embeddings saved at {embedding_path}")
        np.savez(embedding_path, embedding_list=embedding_list, label_list=label_list)
    else:
        logger.info(f"embeddings already exists at {embedding_path}")
    return path


In [39]:
encode_label_names(data_train, "./agnews/label_names.txt", None,"Kermit", "agnews")

encoding class anchor sentences...
generating prompt embeddings using agnews from ./agnews/label_names.txt, template: The news is about <mask>.
embedding path = ./agnews/embeddings\prompt_0_agnews.npz
prompt exampe: The news is about politics.
prompt exampe: The news is about sports.
prompt exampe: The news is about business.
prompt exampe: The news is about technology.
embedding shape: (4, 4000)
generating prompt embeddings using agnews from ./agnews/label_names.txt, template: The news is related to <mask>.
embedding path = ./agnews/embeddings\prompt_1_agnews.npz
prompt exampe: The news is related to politics.
prompt exampe: The news is related to sports.
prompt exampe: The news is related to business.
prompt exampe: The news is related to technology.
embedding shape: (4, 4000)
generating prompt embeddings using agnews from ./agnews/label_names.txt, template: <mask> is the topic of the news.
embedding path = ./agnews/embeddings\prompt_2_agnews.npz
prompt exampe: politics is the topic 

In [45]:
encode_and_save_sentences(data_train, "",",",None, "Kermit", 0.2)

'./agnews/dataset/embeddings\\0.2_simcse_True_Kermit.npz'

In [46]:
print("test")